In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using BenchmarkTools
using TransportMap
using QuadGK
using Distributions
using Random
using LoopVectorization
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative
using Quadrature
using Cubature
using FiniteDiff

### Gradient and Hessian of the log pdf

In [22]:
Nx = 100
Ne = 300
m = 10

X = randn(Nx, Ne) .* randn(Nx, Ne) .+ randn(Nx)
M = HermiteMap(m, X)

@time optimize(M, X, "split"; withqr = true, P = serial)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:20


 20.080367 seconds (50.13 M allocations: 14.071 GiB, 4.61% gc time)


HermiteMap(10, 100, LinearTransform(100, [1.433754397866001, -0.3653698628561821, -0.26801015842888637, -2.391804915074596, 1.2069404472496859, -0.0649450598022652, -1.4934569656361492, -1.7441118889241352, -1.4852464331171922, 1.3096275145099445  …  -0.8083924349873364, -0.41608216235298656, 0.9991300954610142, -0.8251627984234527, -0.6367592017053728, -1.2102146661940822, -0.5267378713660743, 0.4449506212335706, 0.01797707165576713, 1.9706722992905794], [1.1158552714225627 0.0 … 0.0 0.0; 0.0 0.7187654603752137 … 0.0 0.0; … ; 0.0 0.0 … 1.050317781217269 0.0; 0.0 0.0 … 0.0 0.8658817146032199], true), MapComponent[MapComponent(10, 10, 1, IntegratedFunction(10, 10, 1, Rectifier("softplus"), ParametricFunction(ExpandedFunction(10, 10, 1, MultiBasis(Basis of 10 functions: Constant -> 8th degree Probabilistic Hermite function
, 1), [1; 2; … ; 9; 10], [-0.050081689727649065, -1.0736645043091229, -0.8033261535519245, -4.93621327640808, -2.6581769916664255, -5.238931469394311, -3.4922020439439

In [23]:
@time log_pdf(M, X)

  0.068514 seconds (56.43 k allocations: 239.331 MiB, 18.12% gc time)


300-element Array{Float64,1}:
 -110.27320884290553
 -136.16970494239013
 -172.2885208572765
 -162.01056727727922
 -125.40094716901524
 -136.63013936104355
 -150.4358611964215
 -139.33414385600116
 -140.29983992509798
 -116.15630852135585
 -138.16674298437536
 -127.8866320447544
 -165.02375517914115
    ⋮
 -120.29715044780623
 -121.98239237123023
 -121.35821091895662
 -147.268521158218
 -124.50455336048253
 -129.32021949851622
 -116.53214608777611
 -134.65811819375537
 -128.59662395486956
 -126.64305094722869
 -134.1038677939834
 -124.06529961470734

In [24]:
@time grad_x_log_pdf(M, X)

  5.349735 seconds (3.43 M allocations: 26.566 GiB, 12.44% gc time)


300×100 Array{Float64,2}:
  0.901997  -2.23296    -0.39355   …   0.163433    0.165101   -0.289818
  1.16923   -0.0245829  -0.532262     -0.0321478  -1.04023    -0.25955
 -0.266933  -6.40254     0.748527      0.523765   -0.0629691   0.42022
  1.84712   -1.00963     0.943549     -0.558967   -0.288127    0.29482
  4.3853    -1.19015     1.37296      -1.31225     0.656567   -1.488
  0.291751   1.81685     0.826594  …   0.563393    0.018154   -1.00888
 -1.24196   -1.8229     -0.584092      0.655692   -0.528752    0.253719
  1.90308   -0.926737   -0.808776      1.12225     0.242775   -1.19303
 -0.604085  -0.609856   -0.113867      0.952207   -0.0480622  -0.267465
 -0.151359  -1.46716     0.564789     -0.11937    -1.18026    -0.297412
 -0.216217  -0.549059   -2.96553   …  -0.0352656   0.157776   -1.19761
  1.96758   -1.67057    -0.885828     -0.0902416  -0.553684    0.260642
  2.23644   -5.1749     -0.476843     -0.163148   -0.765228    0.807142
  ⋮                                ⋱           

In [27]:
@time hess_x_log_pdf(M.C[70], X[1:70,:])

  3.262501 seconds (8.98 M allocations: 15.879 GiB, 13.53% gc time)


300×70×70 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [ ]:
@time for i=1:Nx
    @show i
    hess_x_log_pdf(M.C[i], X[1:i,:])
end


In [ ]:
@time hess_x_log_pdf(M, X)

i = 1
i = 2
i = 3
i = 

### Integration

In [186]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [187]:
M = HermiteMap(m, X);

In [188]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


 19.530544 seconds (69.35 M allocations: 14.972 GiB, 7.45% gc time)


In [189]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


 10.677751 seconds (44.03 M allocations: 13.718 GiB, 9.93% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed